In [1]:
import requests
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

### 1. Articles scrapping

In [4]:
with open('../.api_auth') as f:
    (api_login, api_pass) = [line.rstrip() for line in f]

ids = []
leads = []
texts = []

date_range = pd.date_range(start="2022-01-01", end="2023-01-31").strftime('%d.%m.%Y').to_list()
for date in tqdm(date_range):
    r1 = requests.get(f'https://api.sta.si/news/sl/{date}', auth=(api_login, api_pass))
    articles_id = eval(r1.content)

    for id in articles_id:
        r2 = requests.get(f'https://api.sta.si/news/sta/{id}', auth=(api_login, api_pass))
        ids.append(id)
        leads.append(r2.json().get('lede', ''))
        texts.append(r2.json().get('text', ''))

dataset = pd.DataFrame({'id': ids, 'lead': leads, 'text': texts})
dataset.to_json(f'../data/raw_articles/01_2022.jsonl', lines=True, orient='records')

100%|██████████| 396/396 [1:00:49<00:00,  9.22s/it]


OSError: Cannot save file into a non-existent directory: '../data/raw_articles'

### 2. Preparing evaluation dataset

In [5]:
dataset.to_json(f'../data/raw_articles/01_2022.jsonl', lines=True, orient='records')

In [12]:
# df = pd.read_json("..\\data\\raw_articles\\all_2023.jsonl", lines=True)
# df_eval = df.sample(200).reset_index(drop=True)
# df_eval.to_json(f'../data/raw_articles/all_2023_eval.jsonl', lines=True, orient='records')

In [13]:
df_eval = pd.read_json("..\\data\\raw_articles\\all_2023_eval.jsonl", lines=True)